In [1]:
import numpy as np
import pandas as pd
import pickle
from calc_auc import calculate_auc
import os

In [95]:
from preprocess_testnew_data import process_testdata
file_path_T = '/home/bioinfo/02_project/01_SHAPE-TM/ML/traing_data/DMS/new_R/test_data_nocontrol/20_HEK293T_human_18s.csv'
depth_threshold = 50
rf_mutation_Count = 0
pipe_truncation_count =0
rate_mut = 0.25
rate_stop =1
acc_threshold = 0
filtered_base =['G','T']
filter_acc = False 
X18s, y18s, y18s_mut_score, y18s_stop_score,df18s_cleaned = process_testdata(file_path_T, depth_threshold, rf_mutation_Count,
                                                             pipe_truncation_count, rate_mut, rate_stop,
                                                             acc_threshold, filtered_bases=filtered_base,
                                                             filter_acc=filter_acc)

X18s.shape,y18s.shape,y18s_mut_score.shape,y18s_stop_score.shape

((914, 15), (914,), (914,), (914,))

In [9]:
from preprocess_testnew_data_control import process_testdata
file_path_T_18s = '/home/bioinfo/02_project/01_SHAPE-TM/ML/traing_data/SHAPE/newR/shape_result/6_7_model_result/SHAPE_data_DMS_model/Control/add/52_ecoli_23s.csv'
file_path_D='/home/bioinfo/02_project/01_SHAPE-TM/ML/traing_data/SHAPE/newR/shape_result/6_7_model_result/SHAPE_data_DMS_model/Control/add/52_DMSO_ecoli_23s.csv'
depth_threshold =50
rf_mutation_Count = 0
pipe_truncation_count =0
rate_mut = 0.25
rate_stop =1
acc_threshold =0
filtered_base =['G','T']
filter_acc = False 
Xsc18s, ysc18s, ysc18s_mut_score, ysc18s_stop_score,dfsc18s_cleaned = process_testdata(file_path_T_18s,file_path_D, depth_threshold, rf_mutation_Count,
                                                             pipe_truncation_count, rate_mut, rate_stop,
                                                             acc_threshold, filtered_bases=filtered_base,
                                                             filter_acc=filter_acc)

Xsc18s.shape,ysc18s.shape,ysc18s_mut_score.shape,ysc18s_stop_score.shape

((1410, 15), (1410,), (1410,), (1410,))

In [2]:
#loadthemodel 
automl1=pickle.load(open("/home/bioinfo/02_project/01_SHAPE-TM/ML/traing_data/DMS/model/DMS_5_24DMS_500s.sav",'rb')) 

In [ ]:
rf_mutation_Count=0
pipe_truncation_count=0
# df=pd.read_csv('/home/bioinfo/02_project/01_SHAPE-TM/ML/traing_data/DMS/new_R/DMS_model_result/DMS_model_5_24_200s/500s/control/SHAPE_nocontrol_AUC.csv')
df = pd.DataFrame(columns=["file_name", "AUC_model", "AUC_mut", "AUC_stop"])
output_directory = '/home/bioinfo/02_project/01_SHAPE-TM/ML/traing_data/DMS/new_R/test_data_nocontrol/only_stop_mut/'
file_name = os.path.basename(file_path_T)
plot_title = file_name.split('.')[0]
save_path=True
save_path=os.path.join(output_directory, f"{plot_title}.pdf")
auc_model,auc_stop,auc_mut,position_SRP,resultSRP,ySRP_mut,ySRP_stop,plt=calculate_auc(X18s, y18s, automl1, df18s_cleaned,rf_mutation_Count,pipe_truncation_count,title=plot_title,save_path=save_path)


new_row = pd.DataFrame({"file_name": [plot_title],  "AUC_model": [auc_model], "AUC_mut": [auc_mut], "AUC_stop": [auc_stop]})
df = pd.concat([df, new_row], ignore_index=True)
#output_csv_path = os.path.join(output_directory, "SHAPE_6_21_nocontrol_onlymut_AUC.csv")
#df.to_csv(output_csv_path, index=False)


from output_model_result import merge_and_save_data
sequence_length_SRP= 1870

csv_path = os.path.join(output_directory, f"{plot_title}.csv")
merge_and_save_data(position_SRP, resultSRP, ySRP_mut,ySRP_stop,sequence_length_SRP, csv_path)

In [ ]:
# calculate no control data
from preprocess_testnew_data import process_testdata
from output_model_result import merge_and_save_data

directory_path = '/home/bioinfo/02_project/01_SHAPE-TM/ML/traing_data/DMS/20_downsampling'
output_directory = '/home/bioinfo/02_project/01_SHAPE-TM/ML/traing_data/DMS/20_downsampling/output'
result_csv_path = os.path.join(output_directory, "20_donwsampling_calc_AUC.csv")
sequence_length = 5300

depth_threshold = 50
rf_mutation_Count = 0
pipe_truncation_count = 0
rate_mut = 0.25
rate_stop = 1
acc_threshold = 0
filtered_base = ['G','T']
filter_acc = False

# load DataFrame
if os.path.exists(result_csv_path):
    df = pd.read_csv(result_csv_path)
else:
    df = pd.DataFrame(columns=["file_name", "AUC_model", "AUC_mut", "AUC_stop"])

for root, dirs, files in os.walk(directory_path):
    for file in files:
        if file.endswith(".csv"):  
            file_path_T = os.path.join(root, file)
            X, y, y_mut_score, y_stop_score, df_cleaned = process_testdata(
                file_path_T, depth_threshold, rf_mutation_Count,
                pipe_truncation_count, rate_mut, rate_stop,
                acc_threshold, filtered_bases=filtered_base,
                filter_acc=filter_acc
            )

            #calc AUC
            file_name = os.path.basename(file_path_T)
            plot_title = file_name.split('.')[0]
            save_path = os.path.join(output_directory, f"{plot_title}.pdf")
            auc_model, auc_stop, auc_mut, position, result, y_mut, y_stop, plt = calculate_auc(
                X, y, automl1, df_cleaned, rf_mutation_Count, pipe_truncation_count, title=plot_title, save_path=save_path
            )

            # 更新结果DataFrame
            new_row = pd.DataFrame({"file_name": [plot_title], "AUC_model": [auc_model], "AUC_mut": [auc_mut], "AUC_stop": [auc_stop]})
            df = pd.concat([df, new_row], ignore_index=True)

            # save result CSV
            df.to_csv(result_csv_path, index=False)

            # save and merge result
            csv_path = os.path.join(output_directory, f"{plot_title}.csv")
            merge_and_save_data(position, result, y_mut, y_stop, sequence_length, csv_path)
